# W02. Park Factors and League Environment

Note: doing rollilng pas. Not sure if calendar date would be better. Has advantages with seasonality, but better hitting weather should be reflected in numerator and denominator even without that, so it might not matter

### Imports

In [1]:
%run "C:\Users\james\Documents\MLB\Code\U1. Imports-WIP.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U2. Utilities.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U3. Classes.ipynb"

In [2]:
%run "C:\Users\james\Documents\MLB\Code\A02. MLB API-WIP.ipynb"

Calculate rolling average for a list of stats in a list <br>
Note: This is NOT shifted!

### Dataset

Calculate Park Factors <br>
Park Factor = Rate at Park (both teams) / Rate in Team's Away Games (both teams) <br>
For example: Fenway Park HR Factor = HR rate at Fenway / HR rate in games where Red Sox are away team

### Period Averages

Average of stats over period of interest, used as base for calculating multipliers

In [3]:
def period_averages(df):
    # Convert to datetime
    df['game_date'] = pd.to_datetime(df['game_date'])

    df = df[df['game_date'] >= '01-01-2015']

    period_avgs = pd.DataFrame(df[events_list].mean()).T

    return period_avgs


### Game Averages

Events that occurred in game

In [4]:
def game_averages(df):
    game_avgs = df.groupby(['gamePk', 'game_date', 'venue_id', 'x_vect', 'y_vect', 'temperature'])[events_list].mean().reset_index()

    return game_avgs

### Player Averages

Averages of stats for players in game

In [5]:
def player_averages(df):
    # Note: these are already shifted if using A02. create_pa_inputs. We want the first PA for players in each game.
    # Stats to average
    batter_inputs_short = [f"{event}_b_long" for event in events_list]
    pitcher_inputs_short = [f"{event}_p_long" for event in events_list]

    # Apply stats from first at bat to entire game
    # First at bat has stats through end of last game
    # This ensures that no stats generated in-game are reflected
    # Note: we're doing this instead of dropping duplicates to properly weight by PA
    df[batter_inputs_short] = df.groupby(['gamePk', 'batter'])[batter_inputs_short].transform('first')
    df[pitcher_inputs_short] = df.groupby(['gamePk', 'pitcher'])[pitcher_inputs_short].transform('first')
    
    # Calculate player averages by game
    batter_avgs = df.groupby(['gamePk'])[batter_inputs_short].mean().reset_index()
    pitcher_avgs = df.groupby(['gamePk'])[pitcher_inputs_short].mean().reset_index()

    # Merge together
    player_avgs = pd.merge(batter_avgs, pitcher_avgs, on='gamePk', how='inner')

    return player_avgs

### League Averages

In [6]:
def league_averages(df, league_window, league_window_min):    
    # Calculate rolling average of stats
    league_avgs = df[events_list].rolling(window=league_window, min_periods=league_window_min).mean()
    league_avgs.columns = [f'{col}_league' for col in league_avgs.columns]

    # Keep column names in a list
    column_names = league_avgs.columns

    # Add game date onto stats
    league_avgs = pd.concat([df[['game_date']], league_avgs], axis=1)

    # Drop duplicates, keeping last
    league_avgs.drop_duplicates('game_date', keep='last', inplace=True)

    # Shift so dates reflect stats through the end of the prior date
    league_avgs[column_names] = league_avgs[column_names].shift(1)

    league_avgs['game_date'] = pd.to_datetime(league_avgs['game_date'])

    
    return league_avgs

### Park Averages

In [7]:
def park_averages(df, park_window, park_window_min):
    # Calculate rolling averages by park 
    park_avgs = df.groupby('venue_id')[events_list].rolling(window=park_window, min_periods=park_window_min).mean()
   
    # Reset index to align with original DataFrame
    park_avgs = park_avgs.reset_index(level=0, drop=False)
    
    # Rename columns to indicate they are park averages
    for column in park_avgs[events_list]:
        park_avgs.rename(columns={column: f"{column}_park"}, inplace=True)

    # Sort to return to correct ordering
    park_avgs.sort_index(ascending=True, inplace=True)
    
    # Add in date
    park_avgs = pd.concat([df[['game_date', 'home_name']], park_avgs], axis=1)

    # Only keep one observation per park
    park_avgs.drop_duplicates(['game_date', 'venue_id'], keep='last', inplace=True)

    
    column_names = [column for columns in park_avgs.columns if "_park" in column]
    
    # Shift so dates reflect stats through the end of the prior date
    park_avgs.groupby(['game_date', 'venue_id'])[column_names].shift(1)
    
    return park_avgs

### Team Averages

In [8]:
def team_averages(df, park_window, park_window_min):
    # Calculate rolling averages by park 
    team_avgs = df.groupby('away_name')[events_list].rolling(window=park_window, min_periods=park_window_min).mean()
   
    # Reset index to align with original DataFrame
    team_avgs = team_avgs.reset_index(level=0, drop=False)
    
    # Rename columns to indicate they are park averages
    for column in team_avgs[events_list]:
        team_avgs.rename(columns={column: f"{column}_team"}, inplace=True)

    # Sort to return to correct ordering
    team_avgs.sort_index(ascending=True, inplace=True)
    
    # Add in date
    team_avgs = pd.concat([df[['game_date']], team_avgs], axis=1)

    # Only keep one observation per park
    team_avgs.drop_duplicates(['game_date', 'away_name'], keep='last', inplace=True)
    
    return team_avgs

### Calculate Park Factors

In [9]:
# # Identify last date in team_avgs before given dates in park_avgs
# latest_dates = []

# for index, row in park_avgs.iterrows():
#     # Filter team_avgs based on criteria
#     filtered_team_avgs = team_avgs[(team_avgs['away_name'] == row['home_name']) & (team_avgs['game_date'] < row['game_date'])]
    
#     # Find the latest date in the filtered dataframe
#     if not filtered_team_avgs.empty:
#         latest_date = filtered_team_avgs['game_date'].max()
#         latest_dates.append(latest_date)
#     else:
#         latest_dates.append(pd.NaT)  # Append NaT if no matching date found

# # Add the latest_dates to park_avgs
# park_avgs['last_road_date'] = latest_dates

# # Merge 
# factor_df = pd.merge(park_avgs, team_avgs, left_on=['home_name', 'last_road_date'], right_on=['away_name', 'game_date'], how='left', suffixes=("", "_"))

# # Loop over event rates and calculate factors
# for event in events_list:
#     factor_df[f'{event}_factor'] = factor_df[f'{event}_park'].astype(float) / factor_df[f'{event}_team'].astype(float)

# factor_df.tail()

### Create Dataset

Calculates Park x Weather multipliers and all necessary components

In [10]:
def create_dataset(df, park_window, park_window_min, league_window, league_window_min, batSide="L"):
    # Only keep regular season games
    df = df[df['game_type_x'] == "R"]
    
    # Only look at one side of the plate
    df = df[df['batSide'] == batSide]
    
    # Reset index
    df.reset_index(inplace=True, drop=True)

    # Create uniform Cleveland name (only necessary for away team, but do both)
    df['away_name'] = np.where(df['away_name'] == "Cleveland Indians", "Cleveland Guardians", df['away_name'])
    df['home_name'] = np.where(df['home_name'] == "Cleveland Indians", "Cleveland Guardians", df['home_name'])

    
    # Convert to datetime
    df['game_date'] = pd.to_datetime(df['game_date'])

    # Convert outputs to numeric (some are boolean)
    df[events_list] = df[events_list].astype('float64')
    

    ### Game Averages
    game_avgs = game_averages(df)

    ### Player Averages
    player_avgs = player_averages(df)


    ### League Averages
    league_avgs = league_averages(df, league_window, league_window_min)


    ### Park Averages
    park_avgs = park_averages(df, park_window, park_window_min)
    park_avgs['home_name'] = np.where(park_avgs['home_name'] == "Cleveland Indians", "Cleveland Guardians", park_avgs['home_name'])

    ### Team Averages
    team_avgs = team_averages(df, park_window, park_window_min)
    team_avgs['away_name'] = np.where(team_avgs['away_name'] == "Cleveland Indians", "Cleveland Guardians", team_avgs['away_name'])


    ### Calculate Park Factors from Park Averages and Team Averages
    # Identify last date in team_avgs before given dates in park_avgs
    # This is so we can identify what the team was doing on the road leading up to their game at home
    latest_dates = []
    
    for index, row in park_avgs.iterrows():
        # Filter team_avgs based on criteria
        filtered_team_avgs = team_avgs[(team_avgs['away_name'] == row['home_name']) & (team_avgs['game_date'] < row['game_date'])]
        
        # Find the latest date in the filtered dataframe
        if not filtered_team_avgs.empty:
            latest_date = filtered_team_avgs['game_date'].max()
            latest_dates.append(latest_date)
        else:
            latest_dates.append(pd.NaT)  # Append NaT if no matching date found

    # Add the latest_dates to park_avgs
    park_avgs['last_road_date'] = latest_dates

    # Merge 
    factor_df = pd.merge(park_avgs, team_avgs, left_on=['home_name', 'last_road_date'], right_on=['away_name', 'game_date'], how='left', suffixes=("", "_"))
    
    # Calculate Park Factors
    for event in events_list:
        factor_df[f'{event}_factor'] = factor_df[f'{event}_park'].astype(float) / factor_df[f'{event}_team'].astype(float)

    # Keep relevant columns
    park_columns = [column for column in factor_df if "_factor" in column] + [column for column in factor_df if "_park" in column] + [column for column in factor_df if "_team" in column]
    keep_columns = ['home_name', 'game_date', 'venue_id'] + park_columns

    factor_df = factor_df[keep_columns]

    factor_df['home_name'] = np.where(factor_df['home_name'] == "Cleveland Indians", "Cleveland Guardian", factor_df['home_name'])

    factor_df['game_date'] = pd.to_datetime(factor_df['game_date'])

    
    ### Merge
    dataset = pd.merge(game_avgs, player_avgs, on='gamePk', how='inner')
    dataset = pd.merge(dataset, league_avgs, on='game_date', how='inner')
    dataset = pd.merge(dataset, factor_df, on=['game_date', 'venue_id'], how='inner')

    # Sort
    dataset.sort_values('game_date', ascending=True, inplace=True)

    # Reset index
    dataset.reset_index(drop=True, inplace=True)
    
    return dataset

Subsets and cleans dataset

In [11]:
def clean_dataset(dataset):
    # Select active ballparks
    active_dataset = dataset[dataset['venue_id'].astype(int).isin(list(team_map['VENUE_ID']))].reset_index(drop=True)
    
    # Restrict to 2015-
    active_dataset = active_dataset[active_dataset['game_date'] >= '01-01-2015']
    
    # Create venue_id dummies
    active_dataset['venue_id_copy'] = active_dataset['venue_id'].copy()
    active_dataset = pd.get_dummies(active_dataset, columns=['venue_id_copy'], drop_first=False, prefix="venue", prefix_sep="_")
    
    # Create interactions of weather and park variables
    weather_interactions = []
    
    for venue in team_map['VENUE_ID']:
        for weather in ['x_vect', 'y_vect', 'temperature']:
            active_dataset[f'venue_{venue}_{weather}'] = active_dataset[f'venue_{venue}'] * active_dataset[weather]
            weather_interactions.append(f'venue_{venue}_{weather}')

    return active_dataset, weather_interactions

### Model

Runs model, training it if selected

In [12]:
def run_model(df, event, batSide, weather_interactions, train=False):
    # Define the dependent variable (y) and independent variables (X)
    y = df.dropna()[f'{event}']

    # Select model inputs
    X_columns = [f'{event}_b_long', f'{event}_p_long', f'{event}_league', f'{event}_factor'] + weather_interactions
    
    # Drop missings
    X = df.dropna()[X_columns]
    # Convert columns to numeric
    X[X_columns] = X[X_columns].astype(float)

    # Add a constant to the independent variables matrix to include an intercept in the model
    X = sm.add_constant(X)
    
    # Add a constant to the independent variables matrix to include an intercept in the model
    if train == True:  
        # Fit the linear regression model
        model = sm.OLS(y, X).fit()

    else:
        # Select model      
        model = globals().get(f'{event}_{str.lower(batSide)}_model')

    return model, X

Apply predictions to dataset

In [13]:
def run_predictions(active_dataset, batSide, weather_interactions, train=False):
    model_dictionary = {}
    for event in events_list:
        print(event)
        # Train model
        model, X = run_model(active_dataset, event, batSide, weather_interactions, train)
        # If we trained a new model,
        if train == True:
            # Save model
            pickle.dump(model, open(os.path.join(model_path, f"Weather Model - {event} {batSide} {todaysdate}"), 'wb'))
            # Save to dictionary
            model_dictionary[event] = model.summary()
    
        # Replace with average
        X[f'{event}_b_long'] = period_avgs[f'{event}'][0]
        X[f'{event}_p_long'] = period_avgs[f'{event}'][0]
    
        # Predict
        X[f'{event}_pred'] = model.predict(X)
        X[f'{event}_mult'] = X[f'{event}_pred'] / period_avgs[f'{event}'][0]
    
        # Copy predicted rate and multiplier to active_dataset
        active_dataset[f'{event}_pred'] = X[f'{event}_pred'].copy()    
        active_dataset[f'{event}_mult'] = X[f'{event}_mult'].copy()

    return active_dataset

### Run

In [14]:
%run "C:\Users\james\Documents\MLB\Code\A02. MLB API-WIP.ipynb"

In [15]:
# Create dataset
complete_dataset = create_pa_inputs(park_factors, team_map, 2015, 2024, short=50, long=300, adjust=False)

In [16]:
# Calculate averages of each stat
period_avgs = period_averages(complete_dataset)
period_avgs.to_csv(os.path.join(baseball_path, "Period Averages.csv"), index=False)

##### LHB

In [17]:
batSide = "L"
dataset_l = create_dataset(complete_dataset, park_window=8000, park_window_min=8000, league_window=200000, league_window_min=200000, batSide=batSide)

In [18]:
active_dataset_l, weather_interactions = clean_dataset(dataset_l)

In [19]:
active_dataset_l = run_predictions(active_dataset_l, batSide, weather_interactions, train=False)

b1
b2
b3
hr
bb
hbp
so
fo
go
lo
po


##### RHB

In [20]:
batSide = "R"
dataset_r = create_dataset(complete_dataset, park_window=8000, park_window_min=8000, league_window=200000, league_window_min=200000, batSide=batSide)

In [21]:
active_dataset_r, weather_interactions = clean_dataset(dataset_r)

In [22]:
active_dataset_r = run_predictions(active_dataset_r, batSide, weather_interactions, train=False)

b1
b2
b3
hr
bb
hbp
so
fo
go
lo
po


### Merge

In [23]:
columns = ['gamePk', 'game_date', 'x_vect', 'y_vect', 'temperature']
league_avg_columns = [column for column in active_dataset_r if "league" in column]
factor_columns = [column for column in active_dataset_r if "factor" in column]
multiplier_columns = [column for column in active_dataset_r if "mult" in column]
venue_columns = [column for column in active_dataset_r if "venue_" in column]

In [24]:
multiplier_dataset = pd.merge(active_dataset_l[columns + venue_columns + league_avg_columns + factor_columns + multiplier_columns], active_dataset_r[columns + league_avg_columns + venue_columns + factor_columns + multiplier_columns], on=columns+venue_columns, how='left', suffixes=("_l", "_r"))

### Save

In [25]:
multiplier_dataset.to_csv(os.path.join(baseball_path, "Multiplier Dataset.csv"), index=False)

In [26]:
multiplier_dataset.tail()

,gamePk,game_date,x_vect,y_vect,temperature,venue_id,venue_1,venue_10,venue_12,venue_14,venue_15,venue_17,venue_19,venue_2,venue_22,venue_2392,venue_2394,venue_2395,venue_2602,venue_2680,venue_2681,venue_2889,venue_3,venue_31,venue_32,venue_3289,venue_3309,venue_3312,venue_3313,venue_4,venue_4169,venue_4705,venue_5,venue_5325,venue_680,venue_7,venue_15_x_vect,venue_15_y_vect,venue_15_temperature,venue_4705_x_vect,venue_4705_y_vect,venue_4705_temperature,venue_2_x_vect,venue_2_y_vect,venue_2_temperature,venue_3_x_vect,venue_3_y_vect,venue_3_temperature,venue_17_x_vect,venue_17_y_vect,venue_17_temperature,venue_4_x_vect,venue_4_y_vect,venue_4_temperature,venue_2602_x_vect,venue_2602_y_vect,venue_2602_temperature,venue_5_x_vect,venue_5_y_vect,venue_5_temperature,venue_19_x_vect,venue_19_y_vect,venue_19_temperature,venue_2394_x_vect,venue_2394_y_vect,venue_2394_temperature,venue_2392_x_vect,venue_2392_y_vect,venue_2392_temperature,venue_7_x_vect,venue_7_y_vect,venue_7_temperature,venue_1_x_vect,venue_1_y_vect,venue_1_temperature,venue_22_x_vect,venue_22_y_vect,venue_22_temperature,venue_4169_x_vect,venue_4169_y_vect,venue_4169_temperature,venue_32_x_vect,venue_32_y_vect,venue_32_temperature,venue_3312_x_vect,venue_3312_y_vect,venue_3312_temperature,venue_3289_x_vect,venue_3289_y_vect,venue_3289_temperature,venue_3313_x_vect,venue_3313_y_vect,venue_3313_temperature,venue_10_x_vect,venue_10_y_vect,venue_10_temperature,venue_2681_x_vect,venue_2681_y_vect,venue_2681_temperature,venue_31_x_vect,venue_31_y_vect,venue_31_temperature,venue_2680_x_vect,venue_2680_y_vect,venue_2680_temperature,venue_680_x_vect,venue_680_y_vect,venue_680_temperature,venue_2395_x_vect,venue_2395_y_vect,venue_2395_temperature,venue_2889_x_vect,venue_2889_y_vect,venue_2889_temperature,venue_12_x_vect,venue_12_y_vect,venue_12_temperature,venue_5325_x_vect,venue_5325_y_vect,venue_5325_temperature,venue_14_x_vect,venue_14_y_vect,venue_14_temperature,venue_3309_x_vect,venue_3309_y_vect,venue_3309_temperature,b1_league_l,b2_league_l,b3_league_l,hr_league_l,bb_league_l,hbp_league_l,so_league_l,fo_league_l,go_league_l,lo_league_l,po_league_l,b1_factor_l,b2_factor_l,b3_factor_l,hr_factor_l,bb_factor_l,hbp_factor_l,so_factor_l,fo_factor_l,go_factor_l,lo_factor_l,po_factor_l,b1_mult_l,b2_mult_l,b3_mult_l,hr_mult_l,bb_mult_l,hbp_mult_l,so_mult_l,fo_mult_l,go_mult_l,lo_mult_l,po_mult_l,b1_league_r,b2_league_r,b3_league_r,hr_league_r,bb_league_r,hbp_league_r,so_league_r,fo_league_r,go_league_r,lo_league_r,po_league_r,b1_factor_r,b2_factor_r,b3_factor_r,hr_factor_r,bb_factor_r,hbp_factor_r,so_factor_r,fo_factor_r,go_factor_r,lo_factor_r,po_factor_r,b1_mult_r,b2_mult_r,b3_mult_r,hr_mult_r,bb_mult_r,hbp_mult_r,so_mult_r,fo_mult_r,go_mult_r,lo_mult_r,po_mult_r
20649,746951,2024-06-11,-2.121320,-2.121320,70,3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,-2.12132,-2.12132,70,-0.0,-0.0,0,-0.0,-0.0,0,-0.000000,-0.000000,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.000000,-0.000000,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,-0.0,-0.0,0,0.137175,0.04293,0.00489,0.029485,0.093045,0.010365,0.22286,0.134735,0.22561,0.05495,0.043955,1.092697,1.400568,1.800000,0.922481,1.007174,1.000000,0.847826,0.856734,1.102876,1.016166,0.997118,1.007176,1.359642,1.327046,0.943327,1.054086,1.081992,0.886387,0.940395,1.070479,0.989450,0.992539,0.14469,0.04445,0.003075,0.030525,0.076885,0.0122,0.227385,0.130295,0.22547,0.05685,0.048175,1.047210,1.025907,1.421053,1.074766,0.944625,0.870690,1.016454,0.835749,1.053404,0.958246,1.072917,1.015552,1.087619,0.843741,0.953983,0.976000,0.966100,1.062254,0.904190,0.999919,0.990571,1.085970
20650,746702,2024-06-11,1.414214,1.414214